In [25]:
import scipy.io
import numpy as np

mat = scipy.io.loadmat('ex4data1.mat')
X = mat['X']
y = mat['y']

#plt.imshow(img_out(X,0))
y_a = np.zeros((10,5000)).astype('int32')
y_temp = np.where(y==10,0,y)
for i in range(5000):
    y_a[y_temp[i],i] = 1
y_a = np.r_[y_a[1:,:], y_a[0,:].reshape(1,-1)].T

In [114]:
X_a = X.reshape(-1,20,20, order='F')
from torch import nn
import torch
from torch.utils.data import DataLoader, TensorDataset, random_split


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

X_b = torch.from_numpy(X_a.astype(np.float32)).to(device)
y_b = (torch.from_numpy(y_a.astype(np.float32))).to(device)
dataset = TensorDataset(X_b, y_b)

test_ratio = 0.2  # 20% data split into test dataset
n_sample = X_a.shape[0]
test_size = int(n_sample * test_ratio)
train_size = n_sample - test_size
train_ds, test_ds = random_split(dataset,[train_size, test_size])
train_dataloader = DataLoader(train_ds, batch_size=train_size, shuffle=True)
test_dataloader = DataLoader(test_ds, batch_size=test_size)
for X_C in train_dataloader:
    X_train = X_C[0].view(-1,1,20,20)
    y_train = X_C[1]
for X_D in test_dataloader:
    X_test = X_D[0].view(-1,1,20,20)
    y_test = X_D[1]

In [117]:
ax = nn.Conv2d(in_channels=1,out_channels=20,kernel_size=5)
ax(X_train).shape

torch.Size([4000, 20, 16, 16])

In [158]:
class CNN_model(nn.Module):
    def __init__(self,input_channel,classes):
        super(CNN_model,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=input_channel, out_channels=20,kernel_size=(5, 5))
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize second set of CONV => RELU => POOL layers
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=(5, 5))
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))
		# initialize first (and only) set of FC => RELU layers
        self.fc1 = nn.Linear(in_features=200, out_features=50)
        self.relu3 = nn.ReLU()
        # initialize our softmax classifier
        self.fc2 = nn.Linear(in_features=50, out_features=classes)
        self.sigmoid = nn.Sigmoid()
    def forward(self,x_val):
        z = self.conv1(x_val)
        z = self.relu1(z)
        z = self.maxpool1(z)
        z = self.conv2(z)
        z = self.relu2(z)
        z = self.maxpool2(z)
        z = torch.flatten(z,1)
        z = self.fc1(z)
        z = self.relu3(z)
        z = self.fc2(z)
        z = self.sigmoid(z)
        return z

In [ ]:
model = CNN_model(1,10)
loss = nn.CrossEntropyLoss()
epoch = int(1e5)

In [181]:
optimizer = torch.optim.Adam(model.parameters(),lr=0.5e-3)
for i in range(epoch):
    out = model(X_train)
    J = loss(out,y_train)
    J.backward()
    optimizer.step()
    optimizer.zero_grad()
    if i%10==0:
        print(f'Epoch number {i} : Loss = {J:.4f}, accuracy {(torch.argmax(out,dim=1)==torch.argmax(y_train)).sum().item()}')

Epoch number 0 : Loss = 1.4690, accuracy 405
Epoch number 10 : Loss = 1.4671, accuracy 406
Epoch number 20 : Loss = 1.4671, accuracy 406
Epoch number 30 : Loss = 1.4670, accuracy 406
Epoch number 40 : Loss = 1.4670, accuracy 405
Epoch number 50 : Loss = 1.4670, accuracy 405
Epoch number 60 : Loss = 1.4670, accuracy 405
Epoch number 70 : Loss = 1.4670, accuracy 405
Epoch number 80 : Loss = 1.4670, accuracy 405
Epoch number 90 : Loss = 1.4670, accuracy 405
Epoch number 100 : Loss = 1.4670, accuracy 405
Epoch number 110 : Loss = 1.4670, accuracy 405


KeyboardInterrupt: 

In [184]:
torch.open

True

In [169]:
(torch.argmax(out,dim=1)==torch.argmax(y_train)).sum().item()

0